# 強化学習とは

```{math}
\newcommand{\E}{{\mathrm E}}
\newcommand{\underE}[2]{\underset{\begin{subarray}{c}#1 \end{subarray}}{\E}\left[ #2 \right]}
\newcommand{\Epi}[1]{\underset{\begin{subarray}{c}\tau \sim \pi \end{subarray}}{\E}\left[ #1 \right]}
```


## 対象とする問題

強化学習は学習主体が制御対象との相互作用をもとに学習し，目標を達成するための枠組みです．
学習主体は **エージェント(Agent)** と呼ばれ，制御対象は **環境(Environment)** と呼ばれます．
制御対象の環境は，一般的に **マルコフ決定過程(Markov decision process; MDP)** で定式化されます．
マルコフ決定過程は$(\mathcal{S}, \mathcal{A}, p, r, p_0)$で定義され，以下に各要素を列挙します．

- 状態空間(State Space): $\mathcal{S} \ni s$
- 行動空間(Action Space): $\mathcal{A} \ni a$
- 状態遷移確率分布関数: $p: \mathcal{S} \times \mathcal{S} \times \mathcal{A} \rightarrow [0, 1]$:

  $p(s'|s, a) := \Pr(S_{t+1} = s'| S_{t} = s, A_t = a),\ \ \ \forall t \in \mathbb{N}_0.$

- 報酬関数(Reward Function): $r: \mathcal{S} \times \mathcal{A} \rightarrow [r_{\text{min}}, r_{\text{max}}]$
- 初期状態確率分布関数(Initial State Distribution): $p_0: \mathcal{S}  \rightarrow [0, 1]:p_0(s) := \Pr(S_0 = s)$

次に，マルコフ決定過程への入力となる行動を決定するルールである **方策(Policy)** 定義します．
方策には様々な形のものがありますが，本書では **確率的方策(Stochastic policy) $\pi : \mathcal{A} \times \mathcal{S} \rightarrow [0,1]$** と **決定的方策(Determinstic policy) $\mu : \mathcal{S} \rightarrow \mathcal{A}$** の 2 種類を取り扱います．
確率的方策は，ある状態のもとである行動を選択する確率を表す関数$\pi(a|s) := \text{Pr}(A=a, S=s)$で表され，
決定的方策は，ある状態のもとで選択する行動を表す関数$a = \mu(s)$で表されます．

これら環境や方策に対して，強化学習は一般的に目的関数と呼ばれる方策を評価する関数を定義し，それを最大化する方策を求める問題と言えます．
目的関数としては典型的に **期待割引累積報酬(expected discounted cumulative reward)** が用いられます．

$$\underE{\pi}{\lim_{T \rightarrow \infty} \sum_{t=0}^{T} \gamma^t R_t } $$

ここで，割引率$\gamma$は$0 < \gamma \geq 1$のパラメータです．マルコフ過程が時間的に無限に続く際，累積報酬を有限の値に収める役割があります．

### プログラムによるシミュレーション

本節では，強化学習が対象とする問題のシミュレーションを行います．
シミュレーションには Python ライブラリの一つである[OpenAI Gym](https://github.com/openai/gym)を用います．
このライブラリには，[様々な環境](https://www.gymlibrary.ml/)が実装されているのですが，今回は基本的な問題の一つである[Cart Pole](https://www.gymlibrary.ml/environments/classic_control/cart_pole/)を用います．

この問題で行動は，0:cart を左に押す/1:cart を右に押す，の二種類で，状態は cart の位置/速度，pole の角度/角速度を表す 4 次元の実数値ベクトルとなります．報酬は pole が倒れていなかったら各時間ステップで 1 得られます．

以下のプログラムは，方策を状態に関わらずランダムとして動かした時の様子の動画と，割引率を 0.99 とした際の割引累積報酬を出力するものです．


In [5]:
import gnwrapper
import gym


env = gnwrapper.LoopAnimation(gym.make("CartPole-v1"))  # 環境を作成

# 状態に関わらずランダムな行動を選択する方策
def policy(state):
    return env.action_space.sample()


gamma = 0.99  # 割引率
r_sum = 0  # 割引累積報酬
t = 0  # 時刻ステップ

state = env.reset()  # 初期化

while True:
    action = policy(state)  # 状態から行動を選択
    next_state, reward, done, info = env.step(action)  # 行動を実行して(次の状態，報酬，終了判定，その他情報)を観測

    r_sum += (gamma**t) * reward

    env.render()  # 描画

    state = next_state  # 状態を更新
    if done:
        break

print(f"Discounted Cumulative Reward : {r_sum}")

env.display()


## 強化学習アルゴリズム

![](../asset/rlalgorithm.png)
*[UCBの授業資料: lecture4](http://rail.eecs.berkeley.edu/deeprlcourse-fa20/])より*

強化学習には様々なアルゴリズムがありますが，一般的には上図のような3ステップを繰り返し，方策を更新していきます．
